<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/btc_prefix_keygen_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
pip install requests

In [ ]:
import requests
import bitcoin

def generate_bitcoin_keypair_with_prefix(prefix):
    counter = 0  # Ajout d'un compteur pour suivre le nombre de clés générées
    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)
        address_uncompressed = bitcoin.pubtoaddr(public_key_uncompressed)

        # Afficher les détails de chaque clé générée
        print(f"--- Tentative #{counter + 1} ---")
        print("Adresse non compressée:", address_uncompressed)

        public_key_compressed = bitcoin.compress(bitcoin.decode_pubkey(public_key_uncompressed, 'hex'))
        address_compressed = bitcoin.pubtoaddr(public_key_compressed)
        print("Adresse compressée:", address_compressed)

        if address_uncompressed.startswith(prefix):
            wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')

            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "public_key_uncompressed": public_key_uncompressed,
                "public_key_compressed": public_key_compressed,
                "address_uncompressed": address_uncompressed,
                "address_compressed": address_compressed
            }

        counter += 1  # Incrémenter le compteur

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)
    data = response.json().get("data", {})

    if not data:
        return 0

    address_data = data.get(address, {})
    if not address_data:
        return 0

    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

kp = generate_bitcoin_keypair_with_prefix("1X")
balance_uncompressed = get_balance(kp["address_uncompressed"])
balance_compressed = get_balance(kp["address_compressed"])

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print("Clé publique non compressée :", kp["public_key_uncompressed"])
print("Clé publique compressée :", kp["public_key_compressed"])
print("Adresse non compressée :", kp["address_uncompressed"], "| Balance:", balance_uncompressed, "BTC")
print("Adresse compressée :", kp["address_compressed"], "| Balance:", balance_compressed, "BTC")
